In [28]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd

In [21]:
Key = '7!H%76ws4g803362Q58473W452b24675'
Secret = '59124Z94T00918XCp143O2l4w0L37g~3'
api_session = '28044854'






In [22]:


# Initialize SDK
breeze = BreezeConnect(api_key=Key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

#print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret=Secret,
                        session_token=api_session)

# Generate ISO8601 Date/DateTime String
import datetime
iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'

In [23]:
#get customer details
breeze.get_customer_details(api_session=api_session) 

{'Success': {'exg_trade_date': {'NSE': '07-Dec-2023',
   'BSE': '07-Dec-2023',
   'FNO': '07-Dec-2023',
   'NDX': '07-Dec-2023'},
  'exg_status': {'NSE': 'C', 'BSE': 'C', 'FNO': 'Y', 'NDX': 'C'},
  'segments_allowed': {'Trading': 'Y',
   'Equity': 'Y',
   'Derivatives': 'D',
   'Currency': 'D'},
  'idirect_userid': 'AG911462',
  'idirect_user_name': 'ANKIT',
  'idirect_ORD_TYP': '',
  'idirect_lastlogin_time': '07-Dec-2023 01:50:10',
  'mf_holding_mode_popup_flg': 'N',
  'commodity_exchange_status': 'N',
  'commodity_trade_date': '06-Dec-2023',
  'commodity_allowed': 'C'},
 'Status': 200,
 'Error': None}

In [31]:
#get holdings
breeze.get_demat_holdings()

{'Success': None, 'Status': 500, 'Error': 'Resource not available.'}

In [42]:
#get_stock details 
ICIBAN = breeze.get_historical_data(interval="1minute",
                            from_date= "2022-08-15T07:00:00.000Z",
                            to_date= "2022-08-17T07:00:00.000Z",
                            stock_code="ITC",
                            exchange_code="NSE",
                            product_type="cash")

In [43]:
type(ICIBAN)
df = pd.DataFrame.from_dict(ICIBAN)
df


,Success,Status,Error
0,"{'datetime': '2022-08-16 09:07:00', 'stock_cod...",200,None
1,"{'datetime': '2022-08-16 09:15:00', 'stock_cod...",200,None
2,"{'datetime': '2022-08-16 09:16:00', 'stock_cod...",200,None
3,"{'datetime': '2022-08-16 09:17:00', 'stock_cod...",200,None
4,"{'datetime': '2022-08-16 09:18:00', 'stock_cod...",200,None
...,...,...,...
786,"{'datetime': '2022-08-17 15:56:00', 'stock_cod...",200,None
787,"{'datetime': '2022-08-17 15:58:00', 'stock_cod...",200,None
788,"{'datetime': '2022-08-17 15:59:00', 'stock_cod...",200,None
789,"{'datetime': '2022-08-17 16:00:00', 'stock_cod...",200,None


In [44]:
#get bank details
breeze.get_funds()

{'Success': {'bank_account': '3223108004024',
  'total_bank_balance': 0.0,
  'allocated_equity': 0.0,
  'allocated_fno': 0.0,
  'block_by_trade_equity': 0.0,
  'block_by_trade_fno': 0.0,
  'block_by_trade_balance': 0.0,
  'unallocated_balance': '0'},
 'Status': 200,
 'Error': None}